# Improving Machine Learning Experience With Multimedia Techniques
Marco A. Franchi

## *Abstract*

*It has been very common face machine learning algorithms at the multimedia area: traffic count; real-time vigilance cameras; baggage tracker; face/expression recognition; and so on. It is so common that it was designed a name for it, Machine Vision. However, it is perceptive the gap between machine learning and multimedia solutions, where even at simple embedded systems it is possible to reach 4k videos running at 60 frames per second, whereas the best neural network solution only handles 224x224 frames at 300 milliseconds in the same system. Due to this, a vast number of solutions as being developed: dedicated hardware for inference process; model manipulation; accelerated pre-processing image solutions; and video manipulation techniques. This paper is based at the study of these videos manipulation techniques, exposing the most common algorithmics, such as frame-skip, frame-droop, resizing, color convert, and overlay solutions; and showing the main difference between them, when the solution can be applied, and the expected performance for each one.*



## Introduction

Aiming to diminish the gap between the machine learning inference process and the multimedia capability, which reaches 4k@60fps, some video manipulation solutions was purposed. Among them, the most common is the overlay solutions, which are able to create alpha layers over the video and insert information on it. These overlays are very common on object detections algorithmics, once they are responsible for drawing a square, select or color an object at the scene. The most common overlays are Scalable Vector Graphics (SVG), Cairo, and OpenCV.
In addition to the overlays, it is important to use a great video framework able to handle all the elements involved at the inference and display solutions. One of the best and most useful ones is the GStreamer framework. GStreamer is able to handle plugins in pipelines, which is perfect to do quick tests.
Apart from the video solutions, it is important to choose the best machine learning algorithmics as well. With a focus on object detection, the most common and valued ones are Single Shot Detection (SSD) and Tensorflow. Both have an incredible inference process capability and the TFlite version demonstrated a great tool for embedded systems.
Thus, with all the tools chosen, this paper intends to compare the combination of theses algorithmics for object detection solutions. This comparison aims to demonstrate how we can increase the video frame rate with simple approaches and demonstrate the best scenarios to handle each neural network algorithmics and the overlays plugins behavior on these tests.

## Materials and Methods

This section describes the material such as the video files, models, labels, and programming language used, and the adopted methodology.

### Programming Language

This paper uses Python 3 language and all the required support for the object detection algorithm: the overlays libs, GStreamer plugins, Tensorflow Lite, and SSD.
* **OpenCV**: Open Source Computer Vision Library, this library is cross-platform and free for user under open-source BSD license. 
* **SVG image**: Scalable Vector Graphics, a Extensible Markup Language (XML)-based vector image format for two-dimensional graphics.  
* **GStreamer**: GStreamer is a library for constructiong graphics of media-handling components and is released under the LGPL license.
* **TFLite**: Tensoflow is a library for dataflow and differenctiable programming across of tasks, free and open-source under the Apache License 2.0. TFLite is a special version for mobile development. 
* **SSD**: Single Shot Detector algorithm designed for object detection in real-time applications.

### Environment

At the paper repository, in addition to the source code, the repository provides a DockerFile for local running. This Docker image provides to run this same experiment with all the required packages and its validated versions.
It is important to alert that this study was made to be performed on embedded Linux systems, including dedicated hardware for inference process and GPU comparison, but it was adapted to run on iterated notebooks, where even basics process such as video playback does not work properly. 

### Models and Labels

As the focus of this paper is the video techniques, and as the SSD and TFlite already have a huge numbers of pre-processed models, this paper will not care about pre-processing or training models and will uses pre-processed models available at the TFLite official object detection support web page.

For this, the following pre-tested model and labels will be used at the tests:

**Model**: mobilenet_ssd_v2_coco_quant_postprocess.tflite

**Labels**: coco_labels.txt

#### MobileNets

MobileNets is a efficient models for mobile and embedded vision applications designed to effectively maximize accuracy while being mindful of the restricted resources for these devices.
#### Common Objects in Context(COCO) Labels

COCO is a large-scale object detection, segmentation, and captioning dataset.

### GStreamer and V4L2

As mentioned before, this paper uses GStreamer framework to reproduce the videos files.
For the comparison purpose, the following approaches will be performed:
* OpenCV V4L2 directly handle;
* GStreamer appsink pipeline + OpenCV V4L2 output;
* GStreamer appsink + appsrc pipelines;
* GStreamer overlay plugins support.

The workflow below describes the difference between each process:

### Workflow

The most basic Object Detection workflow can be described as an input Layer (an image, for instance), being processed by some Hidden Layers (model), and resulting in an Ouput Layer (square over an object and a label on it). So basically a model was used to identify some objects in an image.

However, this basic workflow will only work if the input layer has the same image size and color format expected for the model, i.e, in order to perform an Object Detection, an image pre-processing is required. Also, if it is expected to display the results in the same image, so an image pos-processing is required as well.

To become it more complex, take a video file as source instead of an image, and before considering each frame as an image, the video will require a decoding process, and than the frames will be pre-processed, calculated the results by the model, pos-processed, and than displayed with the label results.

Thus, we can split the Video Object Detection workflow into the following five steps:
* 1- Input data;
* 2- Image Pre-processing; 
* 3- Inference process;
* 4- Image Pos-processing;
* 5- Output data.

**OpenCV V4L2 directly handle:**
![opencv](../data/images/opencv_v4l2.png)
The easiest way to perform Video Object Detection is by using OpenCV to perform the entire workflow process:
* 1 - OpenCV uses Video4Linux (V4L2), a collection of device drivers and an API for supporting realtime video capture on Linux systems, which enable the user to pass a video file, image, or camera as Input Frames. OpenCV will handle it without any improvements.
* 2 - OpenCV will take one-by-one Input Frame, pre-processing it, which includes color convert and image scaling, creating a Pre-processed Frame.
* 3 - Here, the Pre-processed Frame is sent to the model, and then the inference process return the Resulted Frame.
* 4 - The Resulted Frame is manipulated again by the OpenCV, drawing the squares, label, font colors, and etc. Then, the Resulted Frame is again scaled and applied a color convert, returning a Post-processed Frame.
* 5 - In the end, OpenCV uses V4l2 to display the Post-processed Frame at the screen.

The coast of this approach is the impossibility to improve the video capture/display, plus the fact of OpenCV has to handle all the five steps, being overloaded all the time.

The following code demonstrate the OpenCV V4L2 use case:

In [1]:
from ObjectDetection import ObjectsDetectionOpenCV


def main():
    app = ObjectsDetectionOpenCV()
    app.run()


if __name__ == '__main__':
    main()

Inference time: 0:00:00.644234
1.0365520306563747
Inference time: 0:00:00.714231
1.3395297819940328
Inference time: 0:00:00.559280
1.7061393551353534
Inference time: 0:00:00.554750
1.736307117877996
Inference time: 0:00:00.549751
1.778811063035066
Inference time: 0:00:00.548070
1.7830736809551968
Inference time: 0:00:00.523495
1.8591162211019183
Inference time: 0:00:00.518485
1.8563154620714182


**GStreamer appsink pipeline + OpenCV V4L2 output**
![appsink](../data/images/appsink_opencv_v4l2.png)
As mentioned before, one of the issues from the OpenCV approach is the impossibility to improve de video capture. So this new approach will uses a GStreamer pipeline in combination with OpenCV.VideoCapture function in order to improve the video input file. The GStreamer pipeline can be check below:
```
$ gst-launch-1.0 filesrc location=<video_file> ! qtdemux name=demux  demux.video_0 \
! queue ! decodebin ! queue max-size-buffers=1 ! videoconvert ! video/x-raw,format=BGR \ 
! appsink sync=false drop=True max-buffers=1 emit-signals=True max-lateness=8000000000 
```
Altought only the step 1 was changed - and then minus one step to be processed by OpenCV - this change allow us to use some interesting properties, such as dropping frame capability, possibility to select the decoder plugin, and performing the videoconvert by GPU, when supported.

Note that the steps 2 can be avoid as well, by using the videoconvert and videoscale plugins. However, as it only supports CPU, the OpenCV shows a better performance than it.

As results, the displayed framerate will not be impacted for the inference process time or the decoding proces. However, the videoconvert usage, required for the appsink to be able to display the results at screen, is a disavantage, once its only supported by CPU, and resize/color convert by CPU has a high processing coast.

The following code demonstrate this use case:

In [ ]:
from ObjectDetection import ObjectsDetectionV4L2


def main():
    #app = ObjectsDetectionV4L2()
    #app.run()
    print("This method is not supported on docker yet, so the values will be simulated. I am working to solve it.")

if __name__ == '__main__':
    main()

**GStreamer appsink + appsrc pipelines**
![appsink/appsrc](../data/images/appsink_appsrc.png)

Following the idea of GStreamer usage instead of OpenCV, this approach demonstrated the power of appsink and appsrc when used in combination.

By using the same GStreamer appsink pipeline from the last example, but this time sending the Post-processed Frame to another GStreamer pipeline instead of OpenCV, allows us to improve not only the video capture, but the video displayed process as well:

```
$ gst-launch-1.0 filesrc location=<video_file> ! qtdemux name=demux  demux.video_0 \
! queue ! decodebin ! queue max-size-buffers=1 ! videoconvert ! video/x-raw,format=BGR \ 
! appsink sync=false drop=True max-buffers=1 emit-signals=True max-lateness=8000000000 

$ gst-launch-1.0 appsrc name=src is-live=True block=True \
! video/x-raw,format=RGB,width=640,height=480, \
framerate=30/1,interlace-mode=(string)progressive \ 
! videoconvert ! ximagesink
```

Thus, the steps 1 and 5 are being processed by the GStreamer, and the OpenCV will handle only steps 2 and 4.

Again, note that the steps 2 and 4 can be processed by GStreamer as well, but the OpenCV still shows a better performance than videoconvert and videoscale by CPU.

However, on embedded systems, with GPU plugins support to color convert and scaling, this combination shows very promisingly. The properties to be applied are almost unlimited and each one can perform a different result, and even the sink plugin can be one supported by GPU.

The following code demonstrate this use case:

In [ ]:
from ObjectDetection import ObjectsDetectionAppsrc


def main():
    app = ObjectsDetectionAppsrc()
    app.run()
    print("This method is not working properly on docker yet, so the values will be simulated. I am working to solve it.")

if __name__ == '__main__':
    main()

**GStreamer SVG overlay plugins support:**
![overlay](../data/images/gstreamer_overlay.png)

As show before, the image color convert and scaling can be an issue. Due to this, one aditional property can be applyed: overlay. As the name suggest, overlay is the capability to display some plane over other plane, and GStreamer has support for a lot of overlay plugins.

With this approach, and by using rsvgoverlay plugin, this solution is able to take the Input Frame and send it to be processed and displayed at same time. So when it was processed, the overlay, after includes all the image details, as square, labels, etc, will overlay the display image with all the contents:


```
$ filesrc location=<video_file> ! qtdemux name=demux  demux.video_0 \
! queue ! decodebin  ! videorate ! videoconvert n-threads=4 ! videoscale n-threads=4 \
! video/x-raw,width={width},height={height},framerate=30/1 ! queue max-size-buffers=1 leaky=downstream \
! tee name=t 
    t. ! queue max-size-buffers=1 leaky=downstream ! videoconvert ! videoscale \
    ! video/x-raw,width={model_width},height={model_height ! videobox name=box autocrop=true
    ! video/x-raw,format=RGB,width={width},height={height} \
    ! appsink name=appsink emit-signals=true max-buffers=1 drop=true sync=fals
               
    t. ! queue ! videoconvert
    ! rsvgoverlay name=overlay ! videoconvert ! ximagesink
```

Here, the tee usage create two threads: one to be processed by the inference process; other to be displayed. The videobox keeps the frame and is able to return it ot the overlay plugin, so what we see is the combination of two frames, one is the original video, without be touched, other is an alpha image with all the required resizing process being displayed over the original video.

With this approach, the video will never be impacted by the image pre-processing, inference, and image pos-processing steps. So the framerate will be the highest possible.

And note again that all the plugins can be changed by the one with support to GPU, resulting in the best performance possible in terms of framerate.

The following code demonstrate this use case:

In [ ]:
from ObjectDetection import ObjectsDetectionGStreamer

def main():
    app = ObjectsDetectionGStreamer()
    app.run()


if __name__ == '__main__':
    main()

## Results

The examples were made to return the FPS and inference time average collected during the video playback process. With this values, two graphs were generated: one showing the FPS per each example, other showing the Inference time per each example.

The code below generates de values to compose the graphs:

In [1]:
from ObjectDetection import ObjectsDetectionOpenCV
from ObjectDetection import ObjectsDetectionGStreamer

fps = []
inf = []

opencv = ObjectsDetectionOpenCV()
inf_opencv, fps_opencv = opencv.run()

fps.append(fps_opencv)
inf.append(inf_opencv)

#V4L2 is still not working, so I am simulating a value
fps.append(1.5)
inf.append(490)

#APPSRC is still not working, so I am simulating a value
fps.append(2.25)
inf.append(467)

gstreamer = ObjectsDetectionGStreamer()
inf_gstreamer, fps_gstreamer = gstreamer.run()

fps.append(fps_gstreamer)
inf.append(inf_gstreamer)

Inference time: 0:00:00.485371
1.8252011635423793
Inference time: 0:00:00.578293
1.6663129445319897
Inference time: 0:00:00.492800
1.9264456281575226
Inference time: 0:00:00.486227
2.004399086746067
Inference time: 0:00:00.491005
1.9889768951274678
Inference time: 0:00:00.506611
1.9273410279635654
Inference time: 0:00:00.480406
2.0170079556743947
Inference time: 0:00:00.491954
1.9692395604009028
Inference time: 0:00:00.478094
2.0219094144588956
Inference time: 0:00:00.499120
1.9443085027146203
Inference time: 0:00:00.485077
1.9973061173957136
Inference time: 0:00:00.481764
2.0092497096302644
Inference time: 0:00:00.480021
2.020503283364307
Inference time: 0:00:00.484303
2.001761626298084
Inference time: 0:00:00.479576
2.019872356630667
Inference time: 0:00:00.484846
1.9973607673764195
Inference time: 0:00:00.481012
2.0139532805695834
Inference time: 0:00:00.481533
2.0119724715568608
Inference time: 0:00:00.496599
1.9530447496963983
Inference time: 0:00:00.494030
1.9604927461689972
Infe

Inference time: 0:00:00.591369
1.6462598680499998
Inference time: 0:00:00.504584
1.9204603733752528
Inference time: 0:00:00.512238
1.8920737017652658
Inference time: 0:00:00.533133
1.8187075800892936
Inference time: 0:00:00.532505
1.8233403098162602
Inference time: 0:00:00.491917
1.9688866567927132
Inference time: 0:00:00.518651
1.873130047249593
Inference time: 0:00:00.585264
1.680190071300193
Inference time: 0:00:00.553073
1.7545238489400232
Inference time: 0:00:00.544191
1.797076759211723
Inference time: 0:00:00.524900
1.8670537317993183
Inference time: 0:00:00.558334
1.7436583394664917
Inference time: 0:00:00.648329
1.5129616887886554
Inference time: 0:00:00.509827
1.9126410422865445
Inference time: 0:00:00.535715
1.829938973127293
Inference time: 0:00:00.586064
1.6507503981985505
Inference time: 0:00:00.581604
1.6651429637215218
Inference time: 0:00:00.497636
1.9493282606041982
Inference time: 0:00:00.480962
2.012826600640372
Inference time: 0:00:00.484186
1.997735191586121
Infere

Inference time: 0:00:00.581099
1.6729391848463295
Inference time: 0:00:00.550912
1.7572064693475038
Inference time: 0:00:00.577145
1.7016839357126
Inference time: 0:00:00.523855
1.8663177045730044
Inference time: 0:00:00.581774
1.67226138715062
Inference time: 0:00:00.579272
1.6729802291629916
Inference time: 0:00:00.554302
1.7475798282494623
Inference time: 0:00:00.511005
1.9066255822410303
Inference time: 0:00:00.552366
1.7731020525021546
Inference time: 0:00:00.570563
1.6999196130113963
Inference time: 0:00:00.593368
1.6295935943661222
Inference time: 0:00:00.606076
1.6146065523846858
Inference time: 0:00:00.582359
1.6806027399713495
Inference time: 0:00:00.525003
1.8612126104279079
Inference time: 0:00:00.538166
1.8197220077290617
Inference time: 0:00:00.574174
1.6886675506548592
Inference time: 0:00:00.594124
1.6525061159168726
Inference time: 0:00:00.517257
1.8947684233075457
Inference time: 0:00:00.591910
1.6392601816843204
Inference time: 0:00:00.530475
1.8265641702359634
Infer

The code below generates de FPS graph:

In [ ]:
from ObjectDetection import graphs

fps_g = graphs()
fps_g.start(fps, 'FPS')

The code below generates de Inference graph:

In [ ]:
from ObjectDetection import graphs

inf_g = graphs()
inf_g.start(inf, 'Inference*1000 (ms)')

## Conclusion

Unfortunately, the object detection algorithms consume a lot of process resources, and the notebooks have no performance enough to handle it. However, it was still able to show the difference at the inference time and the framerate, both running together and consuming the same CPU process.
According to the values obtained, noticed that the number of about 0.5 milliseconds is far from the expected for a video object detection solution. With some simples calculations, it is easy to note that 0.5 ms of inference time represents 2 frames being calculated per second. It is very slow and it is far from the multimedia capabilities, which can achieve 60 frames per second.

## Source

Martín Abadi, Paul Barham, Jianmin Chen, Zhifeng Chen, Andy Davis, Jeffrey Dean, Matthieu Devin, Sanjay Ghemawat, Geoffrey Irving, Michael Isard, et al. 2016. TensorFlow: A System for Large-Scale Machine Learning.

Ganesh Ananthanarayanan, Paramvir Bahl, Peter Bodík, Krishna Chintalapudi, Matthai Philipose, Lenin Ravindranath, and Sudipta Sinha. 2017. Real-Time Video Analytics: The Killer App for Edge Computing.

Google. 2019. Coral Edge TPU. Retrieved June 16, 2020, from https://coral.ai/

Nvidia. 2020. NVIDIA DeepStream SDK. Retrieved June 16, 2020 from https://developer.nvidia.com/deepstream-sdk

GStreamer. 2019. GStreamer: open source multimedia framework. Retrieved June 16, 2020 from https://gstreamer.freedesktop.org

Intel. 2020.03. Intel's Deep Learning Inference Engine Developer Guide. Retrieved June 16, 2020 from https://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html
